In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.utils import shuffle
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [4]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/reg_metro_rush.csv'
bikeshare_machine = set_data(file)

In [5]:
bikeshare_machine = bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,miles,rush_hour,metro_dist,temp,hum,wind,holiday_1,work_day_1,season_1,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
945796,12.617,1.432146,0,0.141728,13.933349,48.9167,13.916771,0,1,1,...,0,1,0,0,0,0,0,1,0,0
871228,6.717,0.943895,1,0.722611,7.000849,50.8750,9.458993,0,1,1,...,0,0,1,0,0,0,0,0,1,0
530669,25.450,1.615939,0,0.138620,23.999151,47.6250,16.124689,0,1,0,...,0,0,0,0,1,0,0,1,0,0
603645,3.650,0.463464,0,0.048442,2.379151,49.0000,4.458294,0,0,0,...,1,0,0,0,0,0,0,1,0,0
294924,11.417,1.167419,0,0.023451,19.965000,61.4167,16.208975,0,1,0,...,0,0,0,1,0,0,0,1,0,0


### Model 1 - Remove Some Variables that are correlated

In [6]:
# Leave workday, drop weekdays, leave season, drop month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['weekday_0', 'weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6', 
              'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9',
              'month_10','month_11','month_12','time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [7]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25, random_state=17)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((762165, 15), (254056, 15), (762165,), (254056,))

In [8]:
# Fit model
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
pred1 = model1.predict(X1_test)
cvscores_model1 = cross_val_score(model1, X1_train, y1_train, cv=5)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [10]:
cvscores_model1, model1_r, adjustedr1, model1_mse, model1_rmse

(array([ 0.43440672,  0.43370357,  0.43572838,  0.42814748,  0.43895602]),
 0.432489932795034,
 0.43245642369800963,
 36.639360085688857,
 6.0530455215278911)

In [11]:
model1_sm = sm.OLS(y1_train, X1_train.astype(float)).fit()
model1_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.434
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                 4.499e+04
Date:                Fri, 15 Dec 2017   Prob (F-statistic):               0.00
Time:                        17:17:51   Log-Likelihood:            -2.4515e+06
No. Observations:              762165   AIC:                         4.903e+06
Df Residuals:                  762151   BIC:                         4.903e+06
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.2026      0.010    754.461      0.000       7.184       7.221
x2            -1.4757      0.019    -78.303      0.000      -1.513      -1.439
x3            -0.3579      0.031    -11.456      0.000      -0.419      -0.297
x4             0.0600      0.001     41.629      0.000       0.057       0.063
x5            -0.0014      0.001     -2.010      0.044      -0.003   -3.41e-05
x6            -0.0145      0.001     -9.797      0.000      -0.017      -0.012
x7            -0.3863      0.050     -7.682      0.000      -0.485      -0.288
x8            -0.6973      0.017    -40.922      0.000      -0.731      -0.664
x9             1.3449      0.029     46.360      0.000       1.288       1.402
x10            1.9931      0.032     62.288      0.000       1.930       2.056
x11            1.3854      0.034     40.410      0.000       1.318       1.453
x12            1.3882      0.031     45.033      0.000       1.328       1.449
x13            2.3284      0.029     80.075      0.000       2.271       2.385
x14            2.1084      0.037     56.878      0.000       2.036       2.181
x15            1.6749      0.063     26.606      0.000       1.551       1.798
==============================================================================
Omnibus:                   717278.485   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         36491492.680
Skew:                           4.553   Prob(JB):                         0.00
Kurtosis:                      35.652   Cond. No.                     1.44e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.68e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Model 2 - Try reverse of Model 1

In [12]:
# Drop workday, leave weekdays, drop season, leave month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['work_day_1','season_1', 'season_2', 'season_3', 'season_4','time_diff']
X2 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y2 = bikeshare_machine['time_diff']

In [13]:
# Split the data into training and testing sets and check the shape
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.25, random_state=17)
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

((762165, 29), (254056, 29), (762165,), (254056,))

In [14]:
# Fit model
model2 = LinearRegression()
model2.fit(X2_train, y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
pred2 = model2.predict(X2_test)
cvscores_model2 = cross_val_score(model2, X2_train, y2_train, cv=5)
model2_r = model2.score(X2_test, y2_test)
model2_mse = mean_squared_error(y2_test, pred2)
model2_rmse = np.sqrt(model2_mse)
adjustedr2 = 1 - (1-model2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [16]:
cvscores_model2, model2_r, adjustedr2, model2_mse, model2_rmse

(array([ 0.43521129,  0.43424466,  0.43651821,  0.42866985,  0.43967397]),
 0.43321995840028504,
 0.43315525391646692,
 36.592228461831851,
 6.0491510529851915)

In [17]:
# Use statsmodel to check the statistical side of the model
model2_sm = sm.OLS(y2_train, X2_train.astype(float)).fit()
model2_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                 2.256e+04
Date:                Fri, 15 Dec 2017   Prob (F-statistic):               0.00
Time:                        17:18:13   Log-Likelihood:            -2.4511e+06
No. Observations:              762165   AIC:                         4.902e+06
Df Residuals:                  762138   BIC:                         4.903e+06
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.1984      0.010    754.247      0.000       7.180       7.217
x2            -1.4702      0.019    -78.043      0.000      -1.507      -1.433
x3            -0.3523      0.031    -11.280      0.000      -0.413      -0.291
x4             0.0542      0.002     26.231      0.000       0.050       0.058
x5            -0.0044      0.001     -5.982      0.000      -0.006      -0.003
x6            -0.0187      0.002    -12.252      0.000      -0.022      -0.016
x7             0.2203      0.051      4.315      0.000       0.120       0.320
x8             0.2466      0.031      8.020      0.000       0.186       0.307
x9             0.1269      0.026      4.801      0.000       0.075       0.179
x10            0.4908      0.021     23.476      0.000       0.450       0.532
x11            1.2013      0.031     38.363      0.000       1.140       1.263
x12            1.3430      0.029     46.992      0.000       1.287       1.399
x13            0.7118      0.032     22.201      0.000       0.649       0.775
x14            0.3135      0.037      8.479      0.000       0.241       0.386
x15            0.3851      0.033     11.563      0.000       0.320       0.450
x16            0.4119      0.030     13.669      0.000       0.353       0.471
x17            0.4655      0.027     17.352      0.000       0.413       0.518
x18            0.3520      0.029     12.056      0.000       0.295       0.409
x19            0.1389      0.032      4.397      0.000       0.077       0.201
x20            1.2403      0.026     48.023      0.000       1.190       1.291
x21            0.7311      0.025     29.210      0.000       0.682       0.780
x22            0.6742      0.024     28.462      0.000       0.628       0.721
x23            0.5900      0.025     24.069      0.000       0.542       0.638
x24            0.6879      0.024     28.113      0.000       0.640       0.736
x25            0.7613      0.023     33.258      0.000       0.716       0.806
x26            1.5023      0.025     60.955      0.000       1.454       1.551
x27            2.2988      0.032     71.197      0.000       2.236       2.362
x28            2.1111      0.040     52.396      0.000       2.032       2.190
x29            1.7772      0.066     27.084      0.000       1.649       1.906
==============================================================================
Omnibus:                   717497.767   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         36582124.361
Skew:                           4.554   Prob(JB):                         0.00
Kurtosis:                      35.695   Cond. No.                     1.21e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is

### Model 3 - All variables

In [18]:
# Keep all but time_diff
X3 = np.matrix(bikeshare_machine.drop('time_diff', 1))
y3 = bikeshare_machine['time_diff']

In [19]:
# Split the data into training and testing sets and check the shape
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.25, random_state=17)
X3_train.shape, X3_test.shape, y3_train.shape, y3_test.shape

((762165, 34), (254056, 34), (762165,), (254056,))

In [20]:
# Fit model
model3 = LinearRegression()
model3.fit(X3_train, y3_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
pred3 = model3.predict(X3_test)
cvscores_model3 = cross_val_score(model3, X3_train, y3_train, cv=5)
model3_r = model3.score(X3_test, y3_test)
model3_mse = mean_squared_error(y3_test, pred3)
model3_rmse = np.sqrt(model3_mse)
adjustedr3 = 1 - (1-model3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [22]:
cvscores_model3, model3_r, adjustedr3, model3_mse, model3_rmse

(array([ 0.43521259,  0.43425207,  0.43651618,  0.42865573,  0.43966349]),
 0.43322095485720424,
 0.43314509306808102,
 36.592164128975078,
 6.0491457354716687)

In [23]:
# Use statsmodel to check the statistical side of the model
model3_sm = sm.OLS(y3_train, X3_train.astype(float)).fit()
model3_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                 2.023e+04
Date:                Fri, 15 Dec 2017   Prob (F-statistic):               0.00
Time:                        17:18:38   Log-Likelihood:            -2.4511e+06
No. Observations:              762165   AIC:                         4.902e+06
Df Residuals:                  762135   BIC:                         4.903e+06
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.1983      0.010    754.236      0.000       7.180       7.217
x2            -1.4702      0.019    -78.038      0.000      -1.507      -1.433
x3            -0.3523      0.031    -11.282      0.000      -0.414      -0.291
x4             0.0542      0.002     26.192      0.000       0.050       0.058
x5            -0.0044      0.001     -5.913      0.000      -0.006      -0.003
x6            -0.0187      0.002    -12.225      0.000      -0.022      -0.016
x7             0.6758      0.040     16.689      0.000       0.596       0.755
x8             0.4571      0.021     21.633      0.000       0.416       0.498
x9             0.9235      0.034     27.288      0.000       0.857       0.990
x10            0.9880      0.031     31.610      0.000       0.927       1.049
x11            1.0141      0.034     30.183      0.000       0.948       1.080
x12            0.9408      0.037     25.338      0.000       0.868       1.014
x13            0.0964      0.041      2.328      0.020       0.015       0.178
x14           -0.0230      0.038     -0.599      0.549      -0.098       0.052
x15            0.3169      0.029     10.953      0.000       0.260       0.374
x16            0.9872      0.039     25.175      0.000       0.910       1.064
x17            1.1286      0.037     30.470      0.000       1.056       1.201
x18            0.4892      0.035     13.921      0.000       0.420       0.558
x19            0.0736      0.045      1.619      0.105      -0.015       0.163
x20            0.1452      0.043      3.399      0.001       0.061       0.229
x21            0.1931      0.035      5.479      0.000       0.124       0.262
x22            0.2984      0.042      7.171      0.000       0.217       0.380
x23            0.1849      0.043      4.255      0.000       0.100       0.270
x24           -0.0242      0.039     -0.627      0.531      -0.100       0.051
x25            1.2358      0.028     44.319      0.000       1.181       1.290
x26            0.2692      0.018     15.319      0.000       0.235       0.304
x27            0.2122      0.018     11.804      0.000       0.177       0.247
x28            0.1270      0.018      6.869      0.000       0.091       0.163
x29            0.2249      0.018     12.294      0.000       0.189       0.261
x30            0.2995      0.017     17.232      0.000       0.265       0.334
x31            1.4977      0.027     56.151      0.000       1.445       1.550
x32            1.5289      0.020     74.728      0.000       1.489       1.569
x33            1.3394      0.027     49.913      0.000       1.287       1.392
x34            0.9981      0.052     19.208      0.000       0.896       1.100
==============================================================================
Omnibus:                   717486.372   Durbin-Watson:                   2.001
Pr

### Model 4 - Ridge with Round 1

In [24]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge1 = Ridge(fit_intercept=True)
ridge1_gs = GridSearchCV(ridge1, param_grid, cv=5)
ridge1_gs.fit(X1_train, y1_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [25]:
pred4 = ridge1_gs.predict(X1_test)
ridge1_r = ridge1_gs.score(X1_test, y1_test)
ridge1_mse = mean_squared_error(y1_test, pred4)
ridge1_rmse = np.sqrt(ridge1_mse)
adjustedr4 = 1 - (1-ridge1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [26]:
ridge1_r, ridge1_mse, ridge1_rmse, adjustedr4

(0.43248993279578829,
 36.639360085640156,
 6.0530455215238677,
 0.43245642369876391)

### Ridge with Round 2

In [27]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge2 = Ridge(fit_intercept=True)
ridge2_gs = GridSearchCV(ridge2, param_grid, cv=5)
ridge2_gs.fit(X2_train, y2_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [28]:
pred5 = ridge2_gs.predict(X2_test)
ridge2_r = ridge2_gs.score(X2_test, y2_test)
ridge2_mse = mean_squared_error(y2_test, pred5)
ridge2_rmse = np.sqrt(ridge2_mse)
adjustedr5 = 1 - (1-ridge2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [29]:
ridge2_r, ridge2_mse, ridge2_rmse, adjustedr5

(0.43321995840138683,
 36.592228461760719,
 6.0491510529793118,
 0.43315525391756871)

### Ridge with all data

In [30]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge3 = Ridge(fit_intercept=True)
ridge3_gs = GridSearchCV(ridge3, param_grid, cv=5)
ridge3_gs.fit(X3_train, y3_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [31]:
pred6 = ridge3_gs.predict(X3_test)
ridge3_r = ridge3_gs.score(X3_test, y3_test)
ridge3_mse = mean_squared_error(y3_test, pred6)
ridge3_rmse = np.sqrt(ridge3_mse)
adjustedr6 = 1 - (1-ridge3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [32]:
ridge3_r, ridge3_mse, ridge3_rmse, adjustedr6

(0.4332209548577356,
 36.592164128940773,
 6.0491457354688336,
 0.43314509306861249)